<a href="https://colab.research.google.com/github/dung18520632/NhanDien_CS338.L21/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/Assignment3-SentimentAnalysis-with-LSTM.zip

In [27]:
import numpy as np
import os

In [28]:
currentDir = '/content/Assignment3-SentimentAnalysis-with-LSTM'
wordsList = np.load(os.path.join(currentDir, 'wordsList.npy'))
print('Simplified vocabulary loaded!')
wordsList = wordsList.tolist()

wordVectors = np.load(os.path.join(currentDir, 'wordVectors.npy'))
wordVectors = np.float32(wordVectors)
print ('Word embedding matrix loaded!')

Simplified vocabulary loaded!
Word embedding matrix loaded!


In [29]:
print('Size of the vocabulary: ', len(wordsList))
print('Size of the word embedding matrix: ', wordVectors.shape)

Size of the vocabulary:  19899
Size of the word embedding matrix:  (19899, 300)


In [30]:
import tensorflow as tf
maxSeqLength = 10   #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
sentenceIndexes = np.zeros((maxSeqLength), dtype='int32')

# TODO 3.1: Gán chỉ số của các từ trong câu và 'sentenceIndexes'
sentence = 'Món này ăn hoài không hề biết chán'
words = sentence.split(" ")
for index,word in enumerate(words):
    word_idx = wordsList.index(word.lower())
    sentenceIndexes[index]=word_idx
print(sentenceIndexes)
#  Ma trận biểu diễn:
print('Sentence representation of word vectors:')
print(tf.nn.embedding_lookup(wordVectors,sentenceIndexes))

[  119  8136  4884 18791 16614 12231 15951  3371     0     0]
Sentence representation of word vectors:
tf.Tensor(
[[-0.1823 -0.0638  0.2376 ...  0.1462 -0.1092  0.0137]
 [ 0.027  -0.0542  0.1437 ... -0.0913  0.0114  0.0132]
 [ 0.021   0.0102  0.0096 ...  0.411  -0.2519  0.0151]
 ...
 [-0.0239 -0.0383  0.1734 ... -0.0677 -0.096   0.0045]
 [ 0.1882 -0.292   0.0072 ...  0.5919 -0.3094 -0.1228]
 [ 0.1882 -0.292   0.0072 ...  0.5919 -0.3094 -0.1228]], shape=(10, 300), dtype=float32)


In [67]:
sentenceIndexes

array([  119,  8136,  4884, 18791, 16614, 12231, 15951,  3371,     0,
           0], dtype=int32)

In [ ]:
from os import listdir
from os.path import isfile, join
positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
for pf in positiveFiles:
    print(pf)
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)       
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)  
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))

In [40]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [41]:
maxSeqLength = 180

In [43]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
nFiles = 0
# Index of Unknow word
unk_idx = wordsList.index('UNK')

for pf in positiveFiles:
    with open(pf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # TODO 3.2: Nếu 'word' thuộc tập 'wordsList' thì gán chỉ số của 'word' vào ma trận ids
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            # Ngược lại: gán 'unk_idx' vào ma trận ids
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Positive files are indexed!')
for nf in negativeFiles:
    with open(nf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # ToDo 3.2: tương tự như trên. Không khác gì hết.
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Negative files are indexed!')
# Save ids Matrix for future uses.
np.save(os.path.join(currentDir,'idsMatrix.npy'), ids)

IndexError: ignored

In [72]:
ids = np.load(os.path.join(currentDir,'idsMatrix.npy'))
print('Word indexes of the first review: ', ids.shape)

Word indexes of the first review:  (30000, 180)


In [45]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0): 
            # Pick positive samples randomly
            num = randint(1,13999)
            labels.append([1,0])
        else:
            # Pick negative samples randomly
            num = randint(15999,29999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(13999,15999)
        if (num <= 14999):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

In [46]:
numDimensions = 300
batchSize = 64
lstmUnits = 128
nLayers = 2
numClasses = 2
iterations = 30000

In [51]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split

X = ids.copy()
Y = np.concatenate((np.ones((15000,1)),np.zeros((15000,1))))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.067)

In [52]:
checkpoint_path = "/content/training_1/weights.{epoch:04d}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = keras.Sequential([
        layers.Embedding(input_dim=19899,output_dim=numDimensions,weights=[wordVectors],input_length=maxSeqLength,trainable=False),
        layers.LSTM(lstmUnits,return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(lstmUnits),
        layers.Flatten(),
        layers.Dense(2,activation='softmax')]
 )
model.save_weights(checkpoint_path.format(epoch=0))

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='Adam',
    metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batchSize, epochs=30000,callbacks=[cp_callback])

In [54]:
weight_path = '/content/training_1/weights.0053.hdf5'
model.load_weights(weight_path)

In [55]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def Predict_Sentence(sentence,model):
   cleaned = cleanSentences(sentence)
   words = cleaned.split()
   word_vec = np.zeros((1,len(words)))
   for i,word in enumerate(words):
      word_idx = wordsList.index(word)
      word_vec[0][i] = word_idx
   result = model.predict(word_vec)
   if np.argmax(result[0])==1: print('Positive')
   else: print('Negative')

In [65]:
Predict_Sentence('Món ăn này ngon quá đi Món ăn này ngon quá đi',model)

Positive
